## KoNLPy 형태소 분석기 선정

### 대상
1. hannanum
2. kkma
3. okt

### 선정 방법

책 10권에 대해 각각 10개의 키워드 선정
hannanum,kkma,okt 명사추출 기능으로 해당 키워드가 구분되는지 체크 

### 도서목록
1. 리액트를 다루는 기술 = The art of React : 실무에서 알아야 할 기술은 따로 있다!

2. (Do it!) 장고 + 부트스트랩 파이썬 웹 개발의 정석 : 만들면서 배우는 웹 개발 A to Z
3. 머신러닝 디자인 패턴 : 효율적인 머신러닝 파이프라인과 MLOps를 구축하는 30가지 디자인 패턴
4. 핸즈온 머신러닝
5. 파이썬을 이용한 데이터 분석의 정석 : 넘파이, 판다스, 맷플롯립과 실전 예제로 배우는
6. 스파크 완벽 가이드 : 스파크를 활용한 빅데이터 처리와 분석의 모든 것
7. 처음 시작하는 딥러닝
8. 컴퓨터 사이언스 부트캠프 with 파이썬
9. 케라스 창시자에게 배우는 딥러닝
10. 처음 배우는 리액트 네이티브 크로스 플랫폼 앱 개발을 위한 실전 입문서


In [1]:
from konlpy.tag import Hannanum,Okt,Kkma
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec,KeyedVectors
import re

### 성능 비교에 사용 될 도서 10권 추출

In [2]:
bookinfo = pd.read_csv('./data/bookInfo12.csv',encoding='cp949')
testBooksISBN = [9791160508796,9791163032069,9791162244845,9791162242964,9791186710708,9791162241288,9791162243343,9791160504095,9791160505979,9791162243879]
testBooks = bookinfo[bookinfo['col1'].isin(testBooksISBN)]
han = Hannanum()


def mergeListToString(wordList:list) -> str :
    '''
    wordList = item.astype(str).tolist()
    item = pd.Series()
    '''
    str_list = [re.sub('\d','',str(a)) for a in wordList]
    str_list = list(filter(None, str_list))
    result :str = ' '.join(str_list)
    return result

[0.002s][warning][os,thread] Attempt to protect stack guard pages failed (0x00000001693d8000-0x00000001693e4000).
[0.002s][warning][os,thread] Attempt to deallocate stack guard pages failed.


### TF-IDF

In [20]:
# list1 = books.iloc[1].tolist()
def TFIDF(wordsList) :
    from operator import itemgetter

    # str_list = [re.sub('\d','',str(a)) for a in wordList]
    # str_list = list(filter(None, str_list))

    # han = Hannanum()
    # word = ' '.join(str_list)
    # konlpword = han.nouns(word)
    # konlpword = ' '.join(konlpword)

    vectorizer = TfidfVectorizer()
    sp_matrix = vectorizer.fit_transform(wordsList)

    # a,b
    a = np.array(vectorizer.get_feature_names_out())
    b = sp_matrix.toarray()[0]
    unique, counts = np.unique(b, return_counts=True)

    ### Top 10 추출
    k = 0
    for num in range(1,100) :
        if k < 10 :
            k += counts[-num]
        else :
            break

    result =[(a,b) for a,b in zip(a,b) if b >= unique[-num]]            
    result = sorted(result,key=itemgetter(1),reverse=True)
    return result

wordsList=[mergeListToString(i[1]) for i in testBooks.iterrows()]
konlpyWordsForTFIDF = list(map(lambda x : ' '.join(han.nouns(x)),wordsList ))
k = TFIDF(konlpyWordsForTFIDF)
TFIDFKeyword = list(map(lambda x : x[0],k))

### TF-IDF 5000개 적용 및 MultiProcessing 사용

In [ ]:
from operator import itemgetter

#도서별 모든 문장 하나로 합치기
wordsList=[mergeListToString(i[1].astype(str).tolist()) for i in bookinfo.iterrows()] 

 # 모든 문장 형태소 분석
konlpyWordsForTFIDF = list(map(lambda x : ' '.join(han.nouns(x)),wordsList ))


# 형태소 분석 결과 비지도학습
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(konlpyWordsForTFIDF) 

# 결과 추출
a = np.array(vectorizer.get_feature_names_out()) # n개 키워드
b = sp_matrix.toarray() ## 책 권수 만큼 array 생성(5237개). / 개별 array shape = shape(n,)

# Keyword Top 10 추출
result = []
for arr in b :
    unique, counts = np.unique(arr, return_counts=True)
    ### Keyword Top 10 추출
    k = 0
    for num in range(1,100) :
        if k < 10 :
            k += counts[-num]
        else :
            break

    try : 
        itemList =[(name,percent) for name,percent in zip(a,arr) if percent >= unique[-num]]            
        itemList = sorted(itemList,key=itemgetter(1),reverse=True)
        result.append(itemList)
    except:
        result.append(None)

# 추출 결과를 column에 넣기 위한 전처리
TFIDFKeyword =[]
for item in result:
    list1=[]
    for x in item:
        list1.append(x[0])
    TFIDFKeyword.append(list1)

In [265]:
## 개별 단어에 Top 10 확보
# TF-IDF 단점 : 매달 TF-IDF를 돌려야함. 비효율적이라 판단
TFIDFKeyword[0]

['도커', '컨테이너', '실습', '쿠버네티스', '컴포즈', '매니페스트', '커맨드', '파드', '리눅스용', '설치', '실행']

### Word2Vec

### 학습과정

In [7]:
def mergeListToString(item:pd.Series) :
    wordList = item.astype(str).tolist()
    str_list = [re.sub('\d','',str(a)) for a in wordList]
    str_list = list(filter(None, str_list))
    result = ' '.join(str_list)
    return result

wordsList=[mergeListToString(i[1]) for i in bookinfo.iterrows()]
print('Complete wordsList Load!!')
konlpyWords = list(map(lambda x : han.nouns(x),wordsList ))
embedding_model = Word2Vec(sentences=konlpyWords, window = 2, min_count=50, workers=7, sg=1)

Complete wordsList Load!!


### 학습된 vec 열기

In [3]:
# embedding_model.wv.sa ve_word2vec_format('booksTest1') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("booksTest1") # 모델 로드

In [16]:
import ast
kkk = pd.read_csv('./data/keywordExtraction.csv',encoding='cp949')
lists = kkk['keyword'].tolist()

def changeStringToList(strList) :
    strList = strList.replace(' ',', ')
    result = np.array(ast.literal_eval(strList))
    
    return ast.literal_eval(strList)

result = list(map(changeStringToList,lists))

# embedding_model = Word2Vec(sentences=result, window = 2, min_count=50, workers=7, sg=0)


In [51]:
# 중복값 찾기
def findOverlapNum(keywordsOfBook:list,keywordsWord2Vec):
    return sum(np.in1d(keywordsOfBook,keywordsWord2Vec))



searchedKeyword = ['자연어','파이썬']
keywordsWord2Vec = loaded_model.most_similar(positive=searchedKeyword,topn=20)
sums = np.array(list(map(lambda x : findOverlapNum(x,keywordsWord2Vec) ,result)))
kkkk= kkk.copy()
kkkk['sums']=sums

In [52]:
kkkk[kkkk['sums'] > 2][['도서명','ISBN']].head(5)

,도서명,ISBN
127,"딥러닝 텐서플로 교과서 :기초부터 CNN, RNN, 시계열 분석, 성능 최적화, 자...",9791165215477
202,(기초부터 배우는) 인공지능,9788931582819
305,"머신 러닝을 위한 수학 with 파이썬, R - 딥러닝에 필요한 수학만 골라 담았다!",9791165212537
346,자연어 처리 쿡북 with 파이썬,9791161752655
478,"머신 러닝 교과서 with 파이썬, 사이킷런, 텐서플로",9791160507966
